In [1]:
using Laplacians

objc[24802]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[24802]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[24802]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[24802]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_7/"

"../graphs/se_7/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [9]:
@time F = treeSolver(tree)
@time F(b);

  0.219588 seconds (282.53 k allocations: 14.635 MB, 1.90% gc time)
  0.117128 seconds (275.67 k allocations: 8.071 MB)


In [ ]:
# a run on standard Float64 data types
numIts = 50000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 1.4523703083004403e9
Working on iteration 20 with a-norm error 7.559202718955503e8
Working on iteration 30 with a-norm error 4.584467916313619e8
Working on iteration 40 with a-norm error 3.2634833167261493e8
Working on iteration 50 with a-norm error 2.451715532704412e8
Working on iteration 60 with a-norm error 1.9393809587319046e8
Working on iteration 70 with a-norm error 1.714076698529659e8
Working on iteration 80 with a-norm error 1.4518047346286696e8
Working on iteration 90 with a-norm error 1.3007363173704244e8
Working on iteration 100 with a-norm error 1.1806222108261678e8
Working on iteration 110 with a-norm error 1.0566726159296687e8
Working on iteration 120 with a-norm error 9.717118261572142e7
Working on iteration 130 with a-norm error 8.827777142490146e7
Working on iteration 140 with a-norm error 8.107339722681062e7
Working on iteration 150 with a-norm error 7.623557673388042e7
Working on iteration 160 with a-norm error 7.042518208796

In [ ]:
numIts = 50000
for hp in [64,256,4096]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end